In [165]:
import numpy as np

In [166]:
from numpy import random

class Matrix :
    def __init__(self, rows, cols) :
        self.rows = rows
        self.cols = cols
        self.data = np.zeros(shape=(rows,cols))

    # Funções Diversas
    def randomize(self) :
        for row in range(0, self.rows) :
            for col in range(0, self.cols) :
                self.data[row][col] = random.rand(1)
    @staticmethod
    def array_to_matrix(arr) :
        matrix_result = Matrix(len(arr), 1)
        for col in range(0, matrix_result.rows) :
            matrix_result[col] = arr[col]

    # Operações Estáticas Matriz x Matriz
    @staticmethod
    def escalar_multiply(matrix_param, escalar) :
        matrix_result = Matrix(matrix_param.rows, matrix_param.cols)
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_param.data[row][col] * escalar
        return matrix_result
    
    @staticmethod
    def transpose(matrix_param) :
        matrix_result = Matrix(matrix_param.cols, matrix_param.rows)
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[col][row] = matrix_param.data[row][col]
        return matrix_result
    
    @staticmethod
    def hadamard(matrix_a, matrix_b) :
        matrix_result = Matrix(matrix_a.rows, matrix_a.cols) 
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_a.data[row][col] * matrix_b.data[row][col]
        return matrix_result
    
    @staticmethod
    def add(matrix_a, matrix_b) :
        matrix_result = Matrix(matrix_a.rows, matrix_a.cols) 
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_a.data[row][col] + matrix_b.data[row][col]
        return matrix_result
    
    @staticmethod
    def subtract(matrix_a, matrix_b) :
        matrix_result = Matrix(matrix_a.rows, matrix_a.cols) 
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_a.data[row][col] - matrix_b.data[row][col]
        return matrix_result
    
    @staticmethod
    def multiply(matrix_a, matrix_b) :
        matrix_result = Matrix(matrix_a.rows, matrix_a.cols) 
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_a.data[row][col] * matrix_b.data[row][col]
        return matrix_result

In [167]:
import math
def sigmoid(x) : 
    return 1/(1 + math.exp(-x))

def dsigmoid(x) :
    return x * (1-x)

class RNA : 
    def __init__(self, input_size, hidden_size,output_size, learning_rate = 0.1):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.bias_input_hidden = Matrix(self.hidden_size, 1).randomize()
        self.bias_hidden_output = Matrix(self.output_size, 1).randomize()
        
        self.weigths_input_hidden = Matrix(self.hidden_size, 1).randomize()
        self.weigths_hideen_output = Matrix(self.output_size, 1).randomize()
        self.learning_rate = learning_rate

    def train(self, arr, target) : 
        input = Matrix.array_to_matrix(arr)
        hidden = Matrix.multiply(self.weigths_input_hidden, input)
        hidden = Matrix.add(hidden, self.bias_input_hidden)
        map(sigmoid, hidden)
        output = Matrix.multiply(self.weigths_hideen_output, hidden)
        map(sigmoid, output)
    

In [168]:
matrix2 = Matrix(2,2)
matrix1 = Matrix(2,2)

matrix1.data = [[1,2], [3,4]]
matrix2.data = [[1,2], [3,4]]
